# Seattle Collisions - Evaluate Model

The following notebook is for the project model and app evaluation.  This covers the time series piece and mapping.  Main is at the bottom of the notebook where adjustments can be made to.  Note most of the time series evaluation was done in the training workbook because this one was getting too crowded.

In [ ]:
#!conda install -c conda-forge geopy -y
#!conda install -c conda-forge shapely -y

In [128]:
#import branca
#import ipywidgets as widgets

from fbprophet.serialize import model_to_json, model_from_json
from fbprophet.plot import plot_cross_validation_metric 
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation

from geopy.distance import great_circle
from shapely.geometry import MultiPoint

from sklearn.neighbors import BallTree
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import calendar
import json
from math import sqrt
import pandas as pd
import types
import itertools
import numpy as np
from datetime import datetime, timedelta, timezone 
import time

import os
import folium
import folium.plugins

In [ ]:
week = [0,1,2,3,4,5,6]
workweek = [0,1,2,3,4]
weekend = [5,6]
monday = [0]
tuesday = [1]
wednesday = [2]
thursday = [3]
friday = [4]
saturday = [5]
sunday = [6]

monday_color = 'red'
tuesday_color = 'yellow'
wednesday_color = 'orange'
thursday_color = 'blue'
friday_color = 'green'
saturday_color = 'brown'
sunday_color = 'purple'

In [4]:
#df = pd.read_csv('Seattle_Collisions_Final.csv', low_memory = False, parse_dates=True, index_col=0)
#df['INCDTTM'] = pd.to_datetime(df['INCDTTM'])

In [75]:
# Utilities
def get_weekdays(df_source):
    return df_source[(df_source.WEEKDAY == 0)].copy(), df_source[(df_source.WEEKDAY == 1)].copy(), df_source[(df_source.WEEKDAY == 2)].copy(),\
            df_source[(df_source.WEEKDAY == 3)].copy(), df_source[(df_source.WEEKDAY == 4)].copy(), df_source[(df_source.WEEKDAY == 5)].copy(),\
                df_source[(df_source.WEEKDAY == 6)].copy()

def get_workweek_weekend(df_source):
    return df_source[(df_source.WEEKDAY.isin([0,1,2,3,4]))], df_source[(df_source.WEEKDAY.isin([5,6]))]

def get_top(df_source):
    try:
        df_source["XY"] = df_source["X"].astype(str) + "," + df_source["Y"].astype(str)
        return df_source.XY.value_counts().max().astype(int)
    except:
        return 0

def display_top(df_source, largest=5):
    df_source["XY"] = df_source["X"].astype(str) + "," + df_source["Y"].astype(str)
    print(df_source.XY.value_counts().nlargest(largest))
    #print(df_top_5)
    #df_top_5 = pd.DataFrame(list(df_top_5.index.str.split(','))).astype(float)

def map_clusters(map, df_clustered, df_orginal = None, cluster_color = 'red'):
    #map.location = [df_clustered['Y'][0], df_clustered['X'][0]]
    if isinstance(df_orginal, pd.DataFrame):
        df_orginal['XY'] = df_orginal['X'].astype(str) + "," + df_orginal['Y'].astype(str)
        df_top = df_orginal.XY.value_counts().nlargest(3)
        df_top = pd.DataFrame(list(df_top.index.str.split(','))).astype(float)
        for lat, lon in zip(df_top[1], df_top[0]):
            folium.CircleMarker(
                [lat, lon],
                radius=14,
                color=cluster_color,
                fill=True,
                fill_color='yellow',
                fill_opacity=0.5
            ).add_to(map)

    if isinstance(df_clustered, pd.DataFrame):
        for lat, lon in zip(df_clustered['Y'], df_clustered['X']):
            folium.CircleMarker(
                [lat, lon],
                radius=7,
                color=cluster_color,
                fill=True,
                fill_color='orange',
                fill_opacity=0.5
            ).add_to(map)
    else:
        print('No cluster to map: ' + cluster_color)

In [76]:
# Inspired from Geoff Boeing's awesome post.  Used initially workout clusters and moved here for evaluation purposes
# https://geoffboeing.com/2016/06/mapping-everywhere-ever-been/

# Requires Shapely which usually needs an install
def get_centermost_point(cluster):
    try:
        centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
        centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
        return tuple(centermost_point)
    except:
        print(cluster)
        return None
    
def clusterize(df_source, eps, min_samples_hint = 10, region='NA'):
    try:
        
        if min_samples_hint == 0:
            print('Nothing to cluster due to zero sample size')
            return None
    
        min_samples = min_samples_hint
        # represent points consistently as (latitude-Y, longitude-X)
        columns = list('YX')
        coords = df_source.dropna().loc[:,columns].values
        # Define the number of distance in one radian in miles
        distance_units_per_radian = 3959.87433
        # Define epsilon as .05 miles (not kilometers), to be converted to radians for use by DBSCAN via haversine
        epsilon = eps / distance_units_per_radian
        start_time = time.time()
        
        n_clusters = 0
        while n_clusters == 0:
            #print('Sample: %d' % min_sample_hint)
            db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
            cluster_labels = db.labels_
            n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
            if n_clusters == 0:
                min_samples -= 1

        n_noise = list(cluster_labels).count(-1)
        message = '{} Clusters {:,} - {:,}/{:,} Final/Original Samples'
        print(message.format(region, n_clusters, min_samples, min_samples_hint))

        # turn the clusters in to a pandas series, where each element is a cluster of points
        clusters = pd.Series([coords[cluster_labels==n] for n in range(n_clusters)])
        centermost_points = clusters.map(get_centermost_point)
        # unzip the list of centermost points (lat, lon) tuples into separate lat and lon lists
        lats, lons = zip(*centermost_points)
        # from these lats/lons create a new df of one representative point for each cluster
        rep_points = pd.DataFrame({'lon':lons, 'lat':lats})
        rs = rep_points.apply(lambda row: df_source[(df_source['Y']==row['lat']) & (df_source['X']==row['lon'])].iloc[0], axis=1)
        return rs
    except Exception as e: 
        print(e)
        return None

In [77]:
# Find highest silhouette score then for that score, highest cluster count and sample size.
# Used independently to eval various configs to find clusters within the data is much more configurable than Ball Tree
def best_configuration(df_source, verbose = False):
    silhouette_score = cluster_count = sample_size = sample_size = eps = 0
    cluster_metrics, cluster_counts, clusters_found = dbscan_grid_search(df_source, eps_space = np.arange(0.05, 0.1, 0.1),\
                                                     min_samples_space = np.arange(1, 50, 1), min_clusters = 3, max_clusters = 20)
    if clusters_found == False:
        return 0, 0, clusters_found

    df = pd.DataFrame(cluster_metrics, columns=['eps', 'sample_size', 'cluster_count', 'silhouette_score'])
    df['cluster_detail'] = cluster_counts
    
    silhouette_score = best_silhouette_score(df)
    cluster_count = best_cluster_count(df, silhouette_score)
    sample_size = best_sample_size(df, silhouette_score, cluster_count)
    eps = df[(df.sample_size == sample_size) & (df.cluster_count == cluster_count) & (df.silhouette_score == silhouette_score)]['eps'].min()

    if verbose == True:
        print('Silhouette Coefficient: %0.5f' % silhouette_score)
        print('Cluster Count: %d' % cluster_count)
        print('Sample Size: %d' % sample_size)
        print('Epsilon: %0.2f' % eps)
        #print(df)

    return eps, sample_size, clusters_found

def best_silhouette_score(df):
    return df.silhouette_score.max()

def best_cluster_count(df, score):
    return df[df.silhouette_score == score]['cluster_count'].max()

def best_sample_size(df, score, cluster_count):
    return df[(df.silhouette_score == score) & (df.cluster_count == cluster_count)]['sample_size'].max()  

#print(best_silhouette_score(df_gs))
#print(best_cluster_count(df_gs, best_silhouette_score(df_gs)))
#print(best_sample_size(df_gs, best_silhouette_score(df_gs), best_cluster_count(df_gs, best_silhouette_score(df_gs))))

In [78]:
# More eval type code to find clusters
def dbscan_grid_search(data, eps_space = 0.5, min_samples_space = 5, min_clusters = 0, max_clusters = 10):
    from collections import Counter
    columns = list('YX')
    data = data.loc[:,columns].values
    distance_units_per_radian = 3959.87433
    cluster_metrics = []
    cluster_counts = []
    n_iterations = 0
    cluster_found = False
    try:
        for eps in eps_space:
            for samples in min_samples_space:
                dbscan = DBSCAN(eps = eps/distance_units_per_radian, min_samples = samples, algorithm = 'ball_tree', metric = 'haversine')
                dbscan_result = dbscan.fit_predict(np.radians(data))
                cluster_counter = Counter(dbscan_result)
                n_clusters = sum(abs(np.unique(dbscan_result))) - 1
                #n_clusters = len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
                n_iterations += 1
                if n_clusters >= min_clusters and n_clusters <= max_clusters:
                    cluster_found = True
                    cluster_metrics.append([round(eps, 2), samples, n_clusters, metrics.silhouette_score(data, dbscan.labels_)])
                    cluster_counts.append(cluster_counter)

        return cluster_metrics, cluster_counts, cluster_found
    except Exception as e: 
        print(e)
        return None, None, False

In [156]:
# Helpers for Prophet
def get_day_max_temperature(dfw, target_date):
    return dfw[(dfw.DATE == target_date)]['TEMPERATURE'].max()

def get_day_min_temperature(dfw, target_date):
    return dfw[(dfw.DATE == target_date)]['TEMPERATURE'].min()

def get_day_total_precipitation(dfw, target_date):
    return dfw[(dfw.DATE == target_date)]['PRECIPITATION'].sum()

def get_day_solar_azimuth(dfw, target_date):
    return dfw[(dfw.DATE == target_date)]['SOLARAZIMUTH'].max()

def get_daily_forecast_summary_string(target_day=[0]):
    try:
        return calendar.day_name[target_day[0]] + ' (' + str(get_daily_forecast(target_day)) + ')'
    except:
        print('Forecast not available')
        return calendar.day_name[target_day[0]]

# Todo - just grabs the value based on ordinal, this doesn't line up to what's actually in the DF
def get_daily_forecast(target_day=[0]):
    try:
        return round(int(forecast['yhat'][target_day])) # prediction is a float
    except:
        print('Forecast not available')
        return np.nan

Using Ball Tree here to find the collisions close by.  This is still a work in-progress, minimum neighbors is a hack for now.  

In [81]:
def nearest_neighbors(df_source, nearest_in_miles = 0.01, quantile = .9, minimum_neighbors = 0):
    columns = list('YX')
    coordinates = df_source.loc[:,columns].values
    radius = nearest_in_miles / 3959.87433
    bt = BallTree(np.radians(coordinates), metric='haversine')
    neighbors = bt.query_radius(np.radians(coordinates), r=radius, count_only=True, return_distance=False)
    if np.quantile(neighbors, quantile, interpolation='nearest') > minimum_neighbors:
        return df_source.iloc[np.where(neighbors >= np.quantile(neighbors, quantile, interpolation='nearest'))[0]]
    else:
        if np.max(neighbors) > 10:
            return df_source.iloc[np.where(neighbors >= np.quantile(neighbors, quantile, interpolation='nearest'))[0]]

    return pd.DataFrame()

In [92]:
#nearest_neighbors_forecast(20, dft)
def nearest_neighbors_forecast(forecast_size, df_source, nearest_in_miles = 0.01, quantile = .9, minimum_neighbors = 0, verbose = False):
    try:
        columns = list('YX')
        coordinates = df_source.loc[:,columns].values
        bt = BallTree(np.radians(coordinates), metric='haversine')
        temp_df = pd.DataFrame()
        n_count = 0
        while n_count < forecast_size:
            max_index = nearest_neighbors_max_index(bt, df_source)
            temp_df = temp_df.append(df_source.iloc[max_index], ignore_index=True)
            geohash = df_source.iloc[max_index]['geohash_6']
            df_source = df_source[~(df_source.geohash_6 == geohash)]
            if verbose == True:
                print('Geohash -', geohash)
                print('Max Index -', max_index)
                print('Counter -', n_count)
                print('DF Len -', len(df_source))
            n_count += 1
        return temp_df
    except Exception as e: 
        print(e)
        return pdf.DataFrame()

In [83]:
def nearest_neighbors_max_index(bt, df_source, nearest_in_miles = 0.01, quantile = .9, minimum_neighbors = 0):
    columns = list('YX')
    coordinates = df_source.loc[:,columns].values
    radius = nearest_in_miles / 3959.87433
    neighbors = bt.query_radius(np.radians(coordinates), r=radius, count_only=True, return_distance=False)
    return np.argmax(neighbors)

In [80]:
def plot_map(marker_cluster, days = [0,1,2,3,4,5,6], reduce = True, start_date = '2015-12-31', stop_date = '2020-01-01',\
             with_rain = False, cluster_color = 'red', regional=True, quantile=0.9):
    if with_rain == False:
        query_filter = "INCDTTM > @start_date & INCDTTM < @stop_date & HITPARKEDCAR == 0 & PRECIPITATION == 0 & WEEKDAY in @days"
    else:
        query_filter = "INCDTTM > @start_date & INCDTTM < @stop_date & HITPARKEDCAR == 0 & PRECIPITATION > 0 & WEEKDAY in @days"

    if regional == True:
        df_filtered = df.query(query_filter).groupby('MCCP')
        for name, group in df_filtered:
            if reduce == True:
                group = nearest_neighbors(group, quantile=quantile)
            for index, row in group.iterrows():
                folium.CircleMarker(location=[row['Y'],row['X']],
                                    radius= 10,
                                    color=cluster_color,
                                    fill_color='orange',
                                    fill_opacity=0.5,
                                    fill=True).add_to(marker_cluster)
    else:
        df_filtered = df.query(query_filter)
        if reduce == True:
            df_filtered = nearest_neighbors(df_filtered, quantile=quantile)
        for index, row in df_filtered.iterrows():
            folium.CircleMarker(location=[row['Y'],row['X']],
                                radius= 10,
                                color=cluster_color,
                                fill_color='orange',
                                fill_opacity=0.5,
                                fill=True).add_to(marker_cluster)

In [84]:
def plot_forecast_map(forecast_size, marker_cluster, days = [0,1,2,3,4,5,6], reduce = True, start_date = '2015-12-31', stop_date = '2020-01-01',\
             with_rain = False, cluster_color = 'red', regional=True, quantile=0.9):
    if with_rain == False:
        query_filter = "INCDTTM > @start_date & INCDTTM < @stop_date & HITPARKEDCAR == 0 & PRECIPITATION == 0 & WEEKDAY in @days"
    else:
        query_filter = "INCDTTM > @start_date & INCDTTM < @stop_date & HITPARKEDCAR == 0 & PRECIPITATION > 0 & WEEKDAY in @days"
    
    #df_filtered = df.query(query_filter)
    df_filtered = nearest_neighbors_forecast(forecast_size, df.query(query_filter))
    for index, row in df_filtered.iterrows():
        folium.CircleMarker(location=[row['Y'],row['X']],
                            radius= 10,
                            color=cluster_color,
                            fill_color='orange',
                            fill_opacity=0.5,
                            fill=True).add_to(marker_cluster)

In [159]:
def display_week_left_forecast_right_actual(m, forecast_day = tuesday, rain_flag=False):

    fg_1a = folium.FeatureGroup(name=get_daily_forecast_summary_string(monday), show=True).add_to(m.m1)
    fg_1b = folium.FeatureGroup(name=get_daily_forecast_summary_string(tuesday), show=False).add_to(m.m1)
    fg_1c = folium.FeatureGroup(name=get_daily_forecast_summary_string(wednesday), show=False).add_to(m.m1)
    fg_1d = folium.FeatureGroup(name=get_daily_forecast_summary_string(thursday), show=False).add_to(m.m1)
    fg_1e = folium.FeatureGroup(name=get_daily_forecast_summary_string(friday), show=False).add_to(m.m1)
    fg_1f = folium.FeatureGroup(name=get_daily_forecast_summary_string(saturday), show=False).add_to(m.m1)
    fg_1g = folium.FeatureGroup(name=get_daily_forecast_summary_string(sunday), show=False).add_to(m.m1)

    fg_2a = folium.FeatureGroup(name='Monday', show=True).add_to(m.m2)
    fg_2b = folium.FeatureGroup(name='Tuesday', show=False).add_to(m.m2)
    fg_2c = folium.FeatureGroup(name='Wednesday', show=False).add_to(m.m2)
    fg_2d = folium.FeatureGroup(name='Thursday', show=False).add_to(m.m2)
    fg_2e = folium.FeatureGroup(name='Friday', show=False).add_to(m.m2)
    fg_2f = folium.FeatureGroup(name='Saturday', show=False).add_to(m.m2)
    fg_2g = folium.FeatureGroup(name='Sunday', show=False).add_to(m.m2)

    mc_1a = folium.plugins.MarkerCluster(name="Monday")
    plot_forecast_map(get_daily_forecast(monday), mc_1a, monday, with_rain=rain_flag, cluster_color=monday_color)
    mc_1b = folium.plugins.MarkerCluster(name="Tuesday")
    plot_forecast_map(get_daily_forecast(tuesday), mc_1b, tuesday, with_rain=rain_flag, cluster_color=tuesday_color)
    mc_1c = folium.plugins.MarkerCluster(name="Wednesday")
    plot_forecast_map(get_daily_forecast(wednesday), mc_1c, wednesday, with_rain=rain_flag, cluster_color=wednesday_color)
    mc_1d = folium.plugins.MarkerCluster(name="Thursday")
    plot_forecast_map(get_daily_forecast(thursday), mc_1d, thursday, with_rain=rain_flag, cluster_color=thursday_color)
    mc_1e = folium.plugins.MarkerCluster(name="Friday")
    plot_forecast_map(get_daily_forecast(friday), mc_1e, friday, with_rain=rain_flag, cluster_color=friday_color)
    mc_1f = folium.plugins.MarkerCluster(name="Saturday")
    plot_forecast_map(get_daily_forecast(saturday), mc_1f, saturday, with_rain=rain_flag, cluster_color=saturday_color)
    mc_1g = folium.plugins.MarkerCluster(name="Sunday")
    plot_forecast_map(get_daily_forecast(sunday), mc_1g, sunday, with_rain=rain_flag, cluster_color=sunday_color)

    mc_1a.add_to(fg_1a)
    mc_1b.add_to(fg_1b)
    mc_1c.add_to(fg_1c)
    mc_1d.add_to(fg_1d)
    mc_1e.add_to(fg_1e)
    mc_1f.add_to(fg_1f)
    mc_1g.add_to(fg_1g)
    
    mc_2a = folium.plugins.MarkerCluster(name="Monday")
    plot_map(mc_2a, monday, with_rain=rain_flag, cluster_color=monday_color)
    mc_2b = folium.plugins.MarkerCluster(name="Tuesday")
    plot_map(mc_2b, tuesday, with_rain=rain_flag, cluster_color=tuesday_color)
    mc_2c = folium.plugins.MarkerCluster(name="Wednesday")
    plot_map(mc_2c, wednesday, with_rain=rain_flag, cluster_color=wednesday_color)
    mc_2d = folium.plugins.MarkerCluster(name="Thursday")
    plot_map(mc_2d, thursday, with_rain=rain_flag, cluster_color=thursday_color)
    mc_2e = folium.plugins.MarkerCluster(name="Friday")
    plot_map(mc_2e, friday, with_rain=rain_flag, cluster_color=friday_color)
    mc_2f = folium.plugins.MarkerCluster(name="Saturday")
    plot_map(mc_2f, saturday, with_rain=rain_flag, cluster_color=saturday_color)
    mc_2g = folium.plugins.MarkerCluster(name="Sunday")
    plot_map(mc_2g, sunday, with_rain=rain_flag, cluster_color=sunday_color)

    mc_2a.add_to(fg_2a)
    mc_2b.add_to(fg_2b)
    mc_2c.add_to(fg_2c)
    mc_2d.add_to(fg_2d)
    mc_2e.add_to(fg_2e)
    mc_2f.add_to(fg_2f)
    mc_2g.add_to(fg_2g)

    folium.LayerControl().add_to(m)
    return

# Main

In [161]:
# Debugging, hotwire dataframe
#dft = df[(df.INCDTTM > '2014-12-31') & (df.INCDTTM < '2020-01-01') &\
#         (df.HITPARKEDCAR == 0) & (df.PRECIPITATION >= 0) & (df.WEEKDAY.isin([0,1,2,3,4,5,6]))]

In [105]:
df = pd.read_csv('Seattle_Collisions_Final.csv', low_memory = False, parse_dates=True, index_col=0)
df['INCDTTM'] = pd.to_datetime(df['INCDTTM'])

In [ ]:
df_weather = pd.read_csv('Seattle_Weather_Daily.csv', low_memory = False, parse_dates=True, index_col=0)
df_weather.info()

In [162]:
# Load model from JSON, this is wired up in the training notebook
with open('seattle_collision_model.json', 'r') as fin:
    m = model_from_json(json.load(fin))

# Toggle include history to get the dump what's in the model, model is hardwired for seven days now
future = m.make_future_dataframe(periods=7,freq='D',include_history=False)
# Goal would be to make this dynamic and pull weather from somewhere
future['rain'] = future.apply(lambda x: get_day_total_precipitation(df_weather, x.ds.strftime('%Y-%m-%d')), axis=1)
future['temp'] = future.apply(lambda x: get_day_min_temperature(df_weather, x.ds.strftime('%Y-%m-%d')), axis=1)
future['solar_azimuth'] = future.apply(lambda x: get_day_solar_azimuth(df_weather, x.ds.strftime('%Y-%m-%d')), axis=1)
#future.head(), future.tail()
forecast = m.predict(future)
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,temp,temp_lower,temp_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,yhat
0,2020-01-01,3.827799,11.753966,27.103200,3.827799,3.827799,-0.169110,-0.169110,-0.169110,4.828173,...,-0.169110,-0.169110,-0.169110,0.301710,0.301710,0.301710,-0.848952,-0.848952,-0.848952,19.633146
1,2020-01-02,3.827595,13.196240,28.596972,3.827595,3.827595,-0.318158,-0.318158,-0.318158,4.828173,...,-0.318158,-0.318158,-0.318158,0.624014,0.624014,0.624014,-0.824234,-0.824234,-0.824234,20.811308
2,2020-01-03,3.827392,14.251863,30.154980,3.827392,3.827392,-0.034968,-0.034968,-0.034968,4.828173,...,-0.034968,-0.034968,-0.034968,0.797556,0.797556,0.797556,-0.789467,-0.789467,-0.789467,22.138069
3,2020-01-04,3.827188,9.266671,25.602006,3.827188,3.827188,-0.303253,-0.303253,-0.303253,4.828173,...,-0.303253,-0.303253,-0.303253,-0.516659,-0.516659,-0.516659,-0.745850,-0.745850,-0.745850,17.326394
4,2020-01-05,3.826985,6.665557,22.813987,3.826985,3.826985,-0.357903,-0.357903,-0.357903,4.828173,...,-0.357903,-0.357903,-0.357903,-1.318223,-1.318223,-1.318223,-0.694767,-0.694767,-0.694767,14.646118
5,2020-01-06,3.826781,10.215000,26.471362,3.826781,3.826781,-0.318158,-0.318158,-0.318158,4.828173,...,-0.318158,-0.318158,-0.318158,-0.272679,-0.272679,-0.272679,-0.637748,-0.637748,-0.637748,18.327120
6,2020-01-07,3.826578,14.048594,30.643180,3.826578,3.826578,0.014714,0.014714,0.014714,4.828173,...,0.014714,0.014714,0.014714,0.384282,0.384282,0.384282,-0.576434,-0.576434,-0.576434,21.984719


In [160]:
m = folium.plugins.DualMap(location=(47.6062, -122.3321), zoom_start=12, control_scale=True)
display_week_left_forecast_right_actual(m, rain_flag=True)
m